# Tutorial on Ternary Plots

In [ ]:
import poisson_approval as pa
from fractions import Fraction

## SimplexToProfile

Poisson Approval relies on the package ``python-ternary`` to draw plots on the simplex where points have 3 coordinates summing to 1. The coordinates of a point are denoted ``right``, ``top`` and ``left`` because they are respectively equal to 1 at the right, top and left corners of the triangular figure. They typically represent shares of different types in a profile. In order to draw plots, we need a ``SimplexToProfile`` object that will generate the needed profiles:

In [ ]:
simplex_to_profile = pa.SimplexToProfile(
    pa.ProfileNoisyDiscrete, left_type=('abc', 0.5, 0.01), right_type=('bac', 0.5, 0.01), top_type='c>a~b',
    d_type_fixed_share={('abc', 0.1, 0.01): Fraction(1, 10), ('abc', 0.9, 0.01): Fraction(2, 10)})

The above syntax defines a function ``simplex_to_profile`` that maps a tuple (right, top, left) to a profile defined as:

* The class is ``pa.ProfileNoisyDiscrete``,
* A fixed share 1/10 of voters are of type ``('abc', 0.1, 0.01)``, and a fixed share 1/10 are of type ``('abc', 0.9, 0.01)``,
* The other voters, i.e. a share 8/10, are distributed between ``left_type``, ``right_type`` and ``top_type``, in respective proportions that are given by the input tuple (right, top, left).

For example:

In [ ]:
simplex_to_profile(left=Fraction(11, 80), top=Fraction(52, 80), right=Fraction(17, 80))

## Scale

All the plots require a resolution parameter called ``scale``. For example, if ``scale=10``, then the simplex is divided into cells of diameter 1/10 and the value of the plotted function is computed at the center of each cell. For more information, cf. the documentation of the package ``python-ternary``. In this tutorial, we define a global parameter ``SCALE`` and we will use it for all the plots.

In [ ]:
SCALE = 21

## Number of Equilibria

By default, the function ``ternary_plot_n_equilibria`` computes the ordinal equilibria, i.e. those where all voters having the same ranking cast the same ballot:

In [ ]:
pa.ternary_plot_n_equilibria(
    simplex_to_profile,
    scale=SCALE,
    title='Number of ordinal equilibria')

Using the option ``meth``, you can investigate other kinds of equilibria:

In [ ]:
pa.ternary_plot_n_equilibria(
    simplex_to_profile, 
    scale=SCALE,
    title='Number of group equilibria', 
    meth='analyzed_strategies_group')

Depending on the class of profile, the possible values may be ``analyzed_strategies_ordinal`` (the default), ``analyzed_strategies_group`` (for profiles where a reasonable notion of group is defined, such as ``ProfileNoisyDiscrete``), or ``analyzed_strategies_pure`` (for discrete profiles such as ``ProfileDiscrete`` or ``ProfileTwelve``).

## Winners at Equilibrium

The function ``ternary_plot_n_equilibria`` works similarly:

In [ ]:
pa.ternary_plot_winners_at_equilibrium(
    simplex_to_profile, 
    scale=SCALE,
    title='Winners in ordinal equilibria')

In [ ]:
pa.ternary_plot_winners_at_equilibrium(
    simplex_to_profile, 
    scale=SCALE,
    title='Winners in group equilibria',
    legend_title='Winners', 
    meth='analyzed_strategies_group')

## Winning Frequencies in Fictitious Play or Iterated Voting

By default, the function ``ternary_plot_winning_frequencies`` computes the winning frequencies in fictitious play, with an initialization in sincere strategy, and with all update ratio in 1 / (t + 1):

In [ ]:
pa.ternary_plot_winning_frequencies(simplex_to_profile, scale=SCALE, n_max_episodes=100)

You can change this behavior with the optional parameters of the function:

In [ ]:
pa.ternary_plot_winning_frequencies(
    simplex_to_profile, 
    scale=SCALE, 
    meth='iterated_voting',
    init='random',
    samples_per_point=10, 
    perception_update_ratio=1,
    ballot_update_ratio=1,
    winning_frequency_update_ratio=pa.one_over_t_plus_one,
    n_max_episodes=100, 
    title='Winning frequencies in iterated voting',
    legend_title='Winners', )

## Customize the Plot

Grid and lines:

In [ ]:
figure, tax = pa.ternary_figure()
tax.gridlines_simplex(multiple=0.1)
tax.horizontal_line_simplex(0.1)
tax.left_parallel_line_simplex(0.2)
tax.right_parallel_line_simplex(0.3)
tax.line_simplex((1, 0, 0), (0.2, 0.5, 0.3))

Titles and annotations:

In [ ]:
figure, tax = pa.ternary_figure()
tax.set_title_padded('The title of the figure')
tax.right_corner_label('The right label')
tax.top_corner_label('The top label')
tax.left_corner_label('The left label')
tax.annotate_simplex('An annotation', (0.33, 0.33, 0.33))

## Advanced Intensity Heat Maps

First, define a function that maps a point of the simplex to a number:

In [ ]:
def f(right, top, left):
    return (right**2 + top) / (left + 1)

Then use the method ``heatmap_intensity``:

In [ ]:
figure, tax = pa.ternary_figure(scale=SCALE)
tax.heatmap_intensity(f, 
                      left_label='left',
                      right_label='right',
                      top_label='top')
tax.set_title_padded('An intensity heat map')

## Advanced Candidate Heat Maps

First, define a function that maps a point of the simplex to a array of size 3 (associated with candidates a, b, c):

In [ ]:
def g(right, top, left):
    a = top**.5
    b = left**2
    c = 1 - a - b
    return [a, b, c]

Then use the method ``heatmap_candidates``:

In [ ]:
figure, tax = pa.ternary_figure(scale=SCALE)
tax.heatmap_candidates(g, 
                       left_label='left',
                       right_label='right',
                       top_label='top',
                       legend_title='Candidates',
                       legend_style='palette')
tax.set_title_padded('A candidate heat map')